In [8]:
!pip install datasets
!pip install pandas requests onedrivedownloader


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [12]:
import requests
import io
import pandas as pd

# Import the download function instead of OneDriveDownloader class
from onedrivedownloader import download

# Main dataset is kept here
onedrive_link = "https://indianinstituteofscience-my.sharepoint.com/:u:/g/personal/rishavg_iisc_ac_in/Ecr2JHgh-2VBg3bYhcqFQKYBcpNLNsLEPfGNLAKfdKckfQ"

# Extract the file ID from the OneDrive link - not needed for current download method
file_id = onedrive_link.split('/')[-1]  # Assuming the file ID is the last part of the URL

# Download the file using the download function
# filename should be provided and the file will be automatically saved, you can directly input your local path
download(onedrive_link, filename="all_users_rating.tsv")

'all_users_rating.tsv'

In [13]:
# Read the downloaded file into a DataFrame
import pandas as pd

all_users_rating_df = pd.read_csv("all_users_rating.tsv", sep='\t', on_bad_lines='skip')

In [14]:
all_users_rating_df.shape

(3105370, 15)

Handling missing values:
*   Identify missing values
*   Remove missing values


In [15]:
all_users_rating_df.isnull().sum()

,0
marketplace,0
customer_id,0
review_id,0
product_id,0
product_parent,0
product_title,0
product_category,0
star_rating,4
helpful_votes,4
total_votes,4


In [16]:
# Drop Rows with Any Missing Values
all_users_rating_df.dropna(inplace=True)

all_users_rating_df.shape

(3105184, 15)

In [63]:
# Drop unnecessary columns
columns_to_drop = ['vine', 'marketplace', 'verified_purchase', 'product_parent', 'review_body', 'product_category']
all_users_rating_df = all_users_rating_df.drop(columns=columns_to_drop, errors='ignore')

In [64]:
all_users_rating_df.columns

Index(['customer_id', 'review_id', 'product_id', 'product_title',
       'star_rating', 'helpful_votes', 'total_votes', 'review_headline',
       'review_date'],
      dtype='object')

In [65]:
# Distinct No. of Product Title
distinct_product_count = all_users_rating_df['product_title'].nunique()
print("Distinct No. of Product", distinct_product_count)

# Distinct No. of Customers
distinct_customer_count = all_users_rating_df['customer_id'].nunique()
print("Distinct No. of Customers", distinct_customer_count)

Distinct No. of Product 713665
Distinct No. of Customers 1502265


In [72]:
# Create an df with all the customers having rating count >= 3

# Group by customer_id and count the number of ratings for each customer
customer_rating_counts = all_users_rating_df.groupby('customer_id')['star_rating'].count()

# Filter the customer_rating_counts to include only customers with 3 or more ratings
customers_with_at_least_3_ratings = customer_rating_counts[customer_rating_counts >= 3].index

# Create a new DataFrame containing only the rows where the customer_id is in the customers_with_at_least_3_ratings list
df_filtered_user_rating = all_users_rating_df[all_users_rating_df['customer_id'].isin(customers_with_at_least_3_ratings)]

df_filtered_user_rating.head()

,customer_id,review_id,product_id,product_title,star_rating,helpful_votes,total_votes,review_headline,review_date
3,50732546,RATOTLA3OF70O,0373836635,Colby Conspiracy (Colby Agency),5.0,2.0,2.0,fine author on her A-game,2005-10-14
4,51964897,R1TNWRKIVHVYOV,0262181533,The Psychology of Proof: Deductive Reasoning i...,4.0,0.0,2.0,Execellent cursor examination,2005-10-14
6,53000124,R1KJ6MB7MRSQFF,0805076069,Bait and Switch: The (Futile) Pursuit of the A...,4.0,9.0,11.0,I viewed this the opposite of Publisher's Week...,2005-10-14
12,27925116,R7M06Z88PD7SX,0029148510,Acts of War: Behavior of Men in Battle,4.0,14.0,14.0,Solid Book About Facing Death and Killing in War,2005-10-14
20,50732546,R17026W0TBFZGL,045121692X,"The Last Heiress (Friarsgate Inheritance, Book 4)",5.0,1.0,2.0,superb historical romance,2005-10-14


In [73]:
df_filtered_user_rating.shape

(1591279, 9)

In [74]:
# Remove the review with no helpful upvote
df_filtered_user_rating = df_filtered_user_rating[df_filtered_user_rating['helpful_votes'] < 1]

In [75]:
df_filtered_user_rating.shape

(203540, 9)

In [82]:
# Distinct No. of Product(Book) after filtering
distinct_product_count = df_filtered_user_rating['product_title'].nunique()
print("Distinct no. of Product", distinct_product_count)

# Distinct No. of Customers after filtering
distinct_customer_count = df_filtered_user_rating['customer_id'].nunique()
print("Distinct no. of Customers", distinct_customer_count)

# Distinct No. of reviews after filtering
distinct_review_count = df_filtered_user_rating['review_id'].nunique()
print("Distinct no. of reviews", distinct_review_count)

Distinct no. of Product 83442
Distinct no. of Customers 81797
Distinct no. of reviews 203540


In [84]:
df_filtered_books = df_filtered_user_rating[['product_id', 'product_title']].drop_duplicates()
print('Shape of df_filtered_books: ', df_filtered_books.shape)
df_filtered_books.head()

Shape of df_filtered_books:  (89507, 2)


,product_id,product_title
4,0262181533,The Psychology of Proof: Deductive Reasoning i...
49,0373513194,Kiss of the Blue Dragon (Silhouette Bombshell)
189,1410202984,Dahcotah: Life and Legends of the Sioux
220,0816524718,Navajo Nation Peacemaking: Living Traditional ...
255,1591160529,"Inuyasha, Volume 5"


In [97]:
import io
from google.colab import files # Importing the necessary module

# Convert the DataFrame to CSV format and save it to a file
# Saving the dataframe to a csv file first
df_filtered_user_rating.to_csv('filtered_user_rating.csv', index=False)

# Download the file to local system - needed 03-EDA-Step
files.download('filtered_user_rating.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [96]:
import io
from google.colab import files # Importing the necessary module

# Convert the DataFrame to CSV format and save it to a file
# Saving the dataframe to a csv file first
df_filtered_books.to_csv('filtered_books_data.csv', index=False)

# Download the file to local system - needed 03-EDA-Step
files.download('filtered_books_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download book metadata from source

In [85]:
from onedrivedownloader import download

# Main dataset is kept here
onedrive_link = "https://indianinstituteofscience-my.sharepoint.com/:x:/g/personal/rishavg_iisc_ac_in/EdVMAFu4QJxAnyv3U4Bryx0BGPQT1ZzDKe4CftC6NmhnBQ?e=elTH2h"

# Extract the file ID from the OneDrive link - not needed for current download method
file_id = onedrive_link.split('/')[-1]  # Assuming the file ID is the last part of the URL

# Download the file using the download function
# filename should be provided and the file will be automatically saved, you can directly input your local path
download(onedrive_link, filename="Books_metadata.csv")

'Books_metadata.csv'

In [29]:
import pandas as pd

# Assuming 'Books_Data_Clean.csv' is in the current working directory
books_metadata_df = pd.read_csv('Books_metadata.csv')

# Print some info about the DataFrame
books_metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                1070 non-null   int64  
 1   Publishing Year      1069 non-null   float64
 2   Book Name            1047 non-null   object 
 3   Author               1070 non-null   object 
 4   language_code        1017 non-null   object 
 5   Author_Rating        1070 non-null   object 
 6   Book_average_rating  1070 non-null   float64
 7   Book_ratings_count   1070 non-null   int64  
 8   genre                1070 non-null   object 
 9   gross sales          1070 non-null   float64
 10  publisher revenue    1070 non-null   float64
 11  sale price           1070 non-null   float64
 12  sales rank           1070 non-null   int64  
 13  Publisher            1070 non-null   object 
 14  units sold           1070 non-null   int64  
dtypes: float64(5), int64(4), object(6)
mem

In [87]:
print('Shape of books_metadata_df: ', books_metadata_df.shape)

Shape of books_metadata_df:  (1070, 15)


,index,Publishing Year,Book Name,Author,language_code,Author_Rating,Book_average_rating,Book_ratings_count,genre,gross sales,publisher revenue,sale price,sales rank,Publisher,units sold
0,0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903,genre fiction,34160.0,20496.0,4.88,1,HarperCollins Publishers,7000
1,1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,genre fiction,12437.5,7462.5,1.99,2,HarperCollins Publishers,6250
2,2,2015.0,Go Set a Watchman,Harper Lee,eng,Novice,3.31,138669,genre fiction,47795.0,28677.0,8.69,3,"Amazon Digital Services, Inc.",5500
3,3,2008.0,When You Are Engulfed in Flames,David Sedaris,en-US,Intermediate,4.04,150898,fiction,41250.0,24750.0,7.50,3,Hachette Book Group,5500
4,4,2011.0,Daughter of Smoke & Bone,Laini Taylor,eng,Intermediate,4.04,198283,genre fiction,37952.5,22771.5,7.99,4,Penguin Group (USA) LLC,4750


In [95]:
books_metadata_df.head()

,index,Publishing Year,Book Name,Author,language_code,Author_Rating,Book_average_rating,Book_ratings_count,genre,gross sales,publisher revenue,sale price,sales rank,Publisher,units sold
0,0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903,genre fiction,34160.0,20496.0,4.88,1,HarperCollins Publishers,7000
1,1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,genre fiction,12437.5,7462.5,1.99,2,HarperCollins Publishers,6250
2,2,2015.0,Go Set a Watchman,Harper Lee,eng,Novice,3.31,138669,genre fiction,47795.0,28677.0,8.69,3,"Amazon Digital Services, Inc.",5500
3,3,2008.0,When You Are Engulfed in Flames,David Sedaris,en-US,Intermediate,4.04,150898,fiction,41250.0,24750.0,7.50,3,Hachette Book Group,5500
4,4,2011.0,Daughter of Smoke & Bone,Laini Taylor,eng,Intermediate,4.04,198283,genre fiction,37952.5,22771.5,7.99,4,Penguin Group (USA) LLC,4750


In [94]:
# Merge books_entity_df and book_df based on 'Book Name'
merged_user_rating_book_df = pd.merge(books_metadata_df, df_filtered_user_rating, left_on='Book Name', right_on='product_title', how='inner')

print('Shape of merged_user_rating_book_df: ', merged_user_rating_book_df.shape)

Shape of merged_user_rating_book_df:  (7840, 24)


In [89]:
# Print the merged DataFrame
merged_user_rating_book_df.head()

,index,Publishing Year,Book Name,Author,language_code,Author_Rating,Book_average_rating,Book_ratings_count,genre,gross sales,...,units sold,customer_id,review_id,product_id,product_title,star_rating,helpful_votes,total_votes,review_headline,review_date
0,0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903,genre fiction,34160.0,...,7000,48664925,R3KLJ3J19XTSYG,1565114272,Beowulf,5.0,0.0,0.0,Simply the absolute best,2003-03-14
1,1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,genre fiction,12437.5,...,6250,39451820,R1U0V97RAJCGNK,0930289331,Batman: Year One,5.0,0.0,0.0,The perfect beginning for Batman,2003-09-18
2,1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,genre fiction,12437.5,...,6250,42335285,R2RU600MC7MYG4,0930289331,Batman: Year One,5.0,0.0,0.0,Simply Marvelous,2002-11-11
3,1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,genre fiction,12437.5,...,6250,52966385,R1UQ1TLRT6T8TH,1401207529,Batman: Year One,5.0,0.0,0.0,Origins of the Bat,2002-09-12
4,1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,genre fiction,12437.5,...,6250,41093134,R2BX9CNWL8YWJ1,0446389234,Batman: Year One,5.0,0.0,2.0,the comic book equivalent of great art,2002-03-26


In [91]:
# Distinct No. of Product(Book) after filtering
distinct_product_count = merged_user_rating_book_df['product_title'].nunique()
print("Distinct No. of Product", distinct_product_count)

# Distinct No. of Customers after filtering
distinct_customer_count = merged_user_rating_book_df['customer_id'].nunique()
print("Distinct No. of Customers", distinct_customer_count)

Distinct No. of Product 340
Distinct No. of Customers 6293
